# 1. Install and Import Dependencies

In [1]:
%pip install transformers requests beautifulsoup4 pandas numpy

     ---------------------------------------- 7.4/7.4 MB 2.0 MB/s eta 0:00:00
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
     -------------------------------------- 145.3/145.3 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 268.3/268.3 kB 2.7 MB/s eta 0:00:00
  Using cached tokenizers-0.13.3-cp310-cp310-win_amd64.whl (3.5 MB)
     -------------------------------------- 266.3/266.3 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB 2.2 MB/s eta 0:00:00
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

c:\Users\Bhavya\.conda\envs\SentimentAnalysis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('Fucking hell, I love you bitch', return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits

tensor([[ 1.2131, -0.9287, -1.1254, -0.8288,  1.5778]],
       grad_fn=<AddmmBackward0>)

In [6]:
int(torch.argmax(result.logits))+1

5

# 4. Collect Reviews

In [39]:
link = "file:///C:/Users/Bhavya/Downloads/dataset_easy-twitter-search-scraper_2023-07-15_16-55-35-864.html"

In [40]:
r = requests.get(link)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*#nothingphone2*')
results = soup.find_all('tr', {'class':regex})
reviews = [result.text for result in results]

InvalidSchema: No connection adapters were found for 'file:///C:/Users/Bhavya/Downloads/dataset_easy-twitter-search-scraper_2023-07-15_16-55-35-864.html'

In [38]:
reviews

[]

# 5. Load Reviews into DataFrame and Score

In [13]:
import numpy as np
import pandas as pd

In [14]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [15]:
df['review'].iloc[0]

'Super smooth displayBest configurationBest in this price rangeCamera quality very good'

In [16]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [17]:
sentiment_score(df['review'].iloc[1])

4

In [18]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [19]:
df

,review,sentiment
0,Super smooth displayBest configurationBest in ...,5
1,"As usual, one more good phone from OnePlus. If...",4
2,"Lots of bugs. Video contrast, 5g issue, Ear pi...",2
3,overall a good phone but hard to find a temper...,3
4,Very Nice product,5
5,Good cameraNice designOutstanding battery per...,4
6,Go for it best option for premium segment,5
7,Best part is the phone charge duration and nob...,3
8,Battery drain very fast...agar battery backup ...,1
9,Super Performance,5


In [1]:
df['review'].iloc[1]

NameError: name 'df' is not defined

In [20]:
df['sentiment'].mean()

3.7